# Imports

In [1]:
import cv2
import torch
import ultralytics
from ultralytics import YOLO

In [2]:
torch.cuda.is_available()

False

In [3]:
ultralytics.checks()

Ultralytics YOLOv8.0.123  Python-3.11.2 torch-2.0.1+cpu CPU
Setup complete  (8 CPUs, 15.8 GB RAM, 217.2/475.7 GB disk)


# Setting up the model

In [4]:
# Load a model
model = YOLO("yolov8n-pose.pt")  # This model is pretrained. To setup a empty model use : model = YOLO('yolov8n.yaml')
model.info()


YOLOv8n-pose summary: 250 layers, 3295470 parameters, 0 gradients


(250, 3295470, 0, 0.0)

# Training

The COCO8-Pose dataset is a tiny dataset ment for testing pipelines for errors or as a check before training larger datasets.
It is a good dataset to show how easy it is to train a dataset using ultralytics.

In [5]:
# Train the model
# model.train(data='coco8-pose.yaml', epochs=100, imgsz=640) # TODO glöm inte ta bort kommentaren!!!

# Validate
The re-trained YOLOv8n-pose model accuracy on the COCO8-pose dataset.

In [6]:
# model = YOLO('runs/pose/train/weights/best.pt')  # load a custom model # TODO glöm inte ta bort kommentaren!!!

# Validate the model
# metrics = model.val()  # no arguments needed, dataset and settings remembered

# Predict

In [7]:
# Predict with the model
predict = model.predict('bus.jpg', conf=0.75)
# predict on an image
print(predict)


image 1/1 C:\Users\dherm\python\AI2_exam_assignment\bus.jpg: 640x480 3 persons, 134.2ms
Speed: 3.0ms preprocess, 134.2ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)


[ultralytics.yolo.engine.results.Results object with attributes:

boxes: ultralytics.yolo.engine.results.Boxes object
keypoints: ultralytics.yolo.engine.results.Keypoints object
keys: ['boxes', 'keypoints']
masks: None
names: {0: 'person'}
orig_img: array([[[122, 148, 172],
        [120, 146, 170],
        [125, 153, 177],
        ...,
        [157, 170, 184],
        [158, 171, 185],
        [158, 171, 185]],

       [[127, 153, 177],
        [124, 150, 174],
        [127, 155, 179],
        ...,
        [158, 171, 185],
        [159, 172, 186],
        [159, 172, 186]],

       [[128, 154, 178],
        [126, 152, 176],
        [126, 154, 178],
        ...,
        [158, 171, 185],
        [158, 171, 185],
        [158, 171, 185]],

       ...,

       [[185, 185, 191],
        [182, 182, 188],
        [179, 179, 185],
        ...,
        [114, 107, 112],
        [115, 105, 111],
        [116, 106, 112]],

       [[157, 157, 163],
        [180, 180, 186],
        [185, 186, 190],
  

In [8]:
# Show the image with keypoint names

img = cv2.imread('https://ultralytics.com/images/bus.jpg')

results = model('https://ultralytics.com/images/bus.jpg', conf=0.75)[0]

for result in results:
    for keypoint_index, keypoint in enumerate(result.keypoints.data[0].tolist()):
        cv2.putText(img, str(keypoint_index), (int(keypoint[0]), int(keypoint[1])),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

cv2.imshow('Results', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


image 1/1 C:\Users\dherm\python\AI2_exam_assignment\bus.jpg: 640x480 3 persons, 101.1ms
Speed: 3.5ms preprocess, 101.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)
